# Part 1: Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
full_data = pd.read_csv("loan.csv", low_memory=False)

print("Rows: ", full_data.shape[0], "\t Columns: ", full_data.shape[1])

Rows:  2260668 	 Columns:  145


The data set contains records of ~2.2M clients who borrowed money on the platform between 2007-2018. Information  is stored in 145 columns, some of which are filled only once (e.g. at the time of application or only in the case of a particular event occuring), some updated periodically, with only most recent value present. As the aim of the model is to predict default before the loan is issued, we need to make sure that there are is no data leakage, i.e. that training data consists only of information that is known at the time of application. Unfortunately, in case of many columns there exists ambiguity about the point of time it refers to. Since there is no safe way of resolving it other than reaching to the source, we do not risk compromising the integrity of our models and simply will not include these columns in the training data (more in _columns.xlsm_ file ).         

In [3]:
cols = [
    "addr_state",
    "annual_inc",
    "annual_inc_joint",
    "application_type",
    "disbursement_method",
    "earliest_cr_line",
    "emp_length",
    "home_ownership",
    "initial_list_status",
    "installment",
    "int_rate",
    "issue_d",
    "loan_status",
    "pub_rec_bankruptcies",
    "purpose",
    "sec_app_earliest_cr_line",
    "sub_grade",
    "term",
    "verification_status",
    "verification_status_joint",
]

data = full_data.loc[:, cols]

Defaulted loans constitue 12.5% of the whole data set, furthermore, about 40% of the data points correspond to current loans, which are neither default nor fully paid, as we can infer from the base rates, some of them will default in the future, but as we do not know which ones, they are irrelevant for the purpose of building a model.  

In [4]:
full_data.loan_status.value_counts()

Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: loan_status, dtype: int64

In [5]:
# Label defaulted loans as 1 and fully paid loans as 0.

def_types = [
    "Charged Off",
    "Late (31-120 days)",
    "Does not meet the credit policy. Status:Charged Off",
    "Default",
]
def_mask = (
    (data["loan_status"] == def_types[0])
    | (data["loan_status"] == def_types[1])
    | (data["loan_status"] == def_types[2])
    | (data["loan_status"] == def_types[3])
)
paid_mask = data["loan_status"] == "Fully Paid"

d0 = data[paid_mask]
d0["loan_status"].values[:] = 0
d1 = data[def_mask]
d1["loan_status"].values[:] = 1

data = pd.concat([d1, d0], ignore_index=True)
data.loan_status = pd.to_numeric(data.loan_status)

Columns _issue_d, (sec_app_)earliest_cr_line_ contain dates. As the data was collected over an extended period of time, values in _(sec_app_)earliest_cr_line_ are not directly comparable. Moreover, the end goal of our model is to work with unseen data, hence _issue_d_ must also be sensibly transformed. One way of achieving both is to create new features, namely, _(_sec_app_)yrs_since_last_cr_line_ and _issue_quarter,_ the first two are now directly comparable, whereas the third enables us to take into account seasonal patterns in the data (e.g. borrowers in the holiday season may be more likely to act on impulse, hence more likely to default etc.) 

In [6]:
# create new features and discard old ones
data.issue_d = pd.to_datetime(data.issue_d)
data.earliest_cr_line = pd.to_datetime(data.earliest_cr_line)

data.sec_app_earliest_cr_line = pd.to_datetime(data.sec_app_earliest_cr_line)
data["sec_app_yrs_since_last_cr_line"] = (
    data.issue_d - data.sec_app_earliest_cr_line
) / np.timedelta64(1, "Y")

data["yrs_since_last_cr_line"] = (
    data.issue_d - data.earliest_cr_line
) / np.timedelta64(1, "Y")
data["issue_quarter"] = data.issue_d.dt.quarter

data = data.drop(
    columns=["issue_d", "earliest_cr_line", "sec_app_earliest_cr_line"]
)

We need to deal with non-numerical columns. In our data set there are two types of such data we need to treat separately. For _sub_grade_ and _emp_length_ there exists a natural ordering that we want to preserve, whereas for categories such as _purpose_ this does not hold. For the former we map them to natural numbers according to their order, while for the latter we will use one-hot encoding i.e. each category will become an independent feature taking values in $\{0,1\}$.

In [7]:
# encode ordinal categories
vs_sub_grade = data.sub_grade.unique()
vs_sub_grade.sort()
vs_sub_grade = {vs_sub_grade[i]: i for i in range(len(vs_sub_grade))}
data.sub_grade = data.sub_grade.replace(vs_sub_grade)

vs_emp_length = {
    "2 years": 2,
    "3 years": 3,
    "10+ years": 10,
    "8 years": 8,
    "5 years": 5,
    "6 years": 6,
    "< 1 year": 0,
    "4 years": 4,
    "7 years": 7,
    np.nan: 0,
    "9 years": 9,
    "1 year": 1,
}
data.emp_length = data.emp_length.replace(vs_emp_length)

vs_term = {" 36 months": 3, " 60 months": 5}
data.term = data.term.replace(vs_term)

To avoid adding 50 extra features we group address states into 4 regions as defined by the US Census Bureau. This transformation should be able to preserve information contained in the applicant's location, without drastically increasing the number of features and possibly reducing bias stemming from small amount of data in some locations (e.g. there were only 14 customers from Indiana).

In [8]:
us_regions = {
    "AK": "West",
    "AL": "South",
    "AR": "South",
    "AZ": "West",
    "CA": "West",
    "CO": "West",
    "CT": "Northeast",
    "DC": "South",
    "DE": "South",
    "FL": "South",
    "GA": "South",
    "HI": "West",
    "IA": "Midwest",
    "ID": "West",
    "IL": "Midwest",
    "IN": "Midwest",
    "KS": "Midwest",
    "KY": "South",
    "LA": "South",
    "MA": "Northeast",
    "MD": "South",
    "ME": "Northeast",
    "MI": "Midwest",
    "MN": "Midwest",
    "MO": "Midwest",
    "MS": "South",
    "MT": "West",
    "NC": "South",
    "ND": "Midwest",
    "NE": "Midwest",
    "NH": "Northeast",
    "NJ": "Northeast",
    "NM": "West",
    "NV": "West",
    "NY": "Northeast",
    "OH": "Midwest",
    "OK": "South",
    "OR": "West",
    "PA": "Northeast",
    "RI": "Northeast",
    "SC": "South",
    "SD": "Midwest",
    "TN": "South",
    "TX": "South",
    "UT": "West",
    "VA": "South",
    "VT": "Northeast",
    "WA": "West",
    "WI": "Midwest",
    "WV": "South",
    "WY": "West",
}
data.addr_state = data.addr_state.replace(us_regions)

Some columns have missing values. For _annual_inc_ we fill them with median. Columns related to secondary applicant have missing values only if there is no secondary applicant. Since we do not want them to contribute to the input in the case of an individual application, we fill them with zeros if they are numerical and do not one-hot encode them if they are categorical. For _earliest_cr_line_ and _emp_length_ we make an assumption that if there is no information provided we assign lowest value from the range, as typically being employed for a long time or having a long credit history plays in favour of the applicant, so withholding such information is more probable in the case of the opposite. For _pub_rec_bankruptcies_ and no value probably means that the applicants record is clear. Additionally we will use secondary applicant income as a feature instead of joint income.

In [9]:
for a in sorted(data.columns[data.isna().any()]):
    print(a)

annual_inc_joint
pub_rec_bankruptcies
sec_app_yrs_since_last_cr_line
verification_status_joint
yrs_since_last_cr_line


In [10]:
# fill missing income with median income
data["annual_inc"].fillna(data["annual_inc"].median(), inplace=True)

In [11]:
# calculate secondary applicant income
data["sec_app_annual_inc"] = data["annual_inc_joint"] - data["annual_inc"]
data = data.drop(columns="annual_inc_joint")

In [12]:
# get a list of categorical columns
cat_cols = ["issue_quarter"]
for i in range(len(data.dtypes)):
    if data.dtypes[i] == "object":
        cat_cols.append(data.columns[i])
for name in sorted(cat_cols):
    print(name)

addr_state
application_type
disbursement_method
home_ownership
initial_list_status
issue_quarter
purpose
verification_status
verification_status_joint


In [13]:
# one-hot encode categorical variables and fill remaining with zeros
data = pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)
data.fillna(0, inplace=True)

In [14]:
# save to file
data.to_csv("preprocessed2.csv", index=False)